# RBM emulator for coupled channels scattering

We would like to construct an emulator for a toy problem for inelastic nucleon-nucleus scattering in which we explicitly include a few excited states of the target in our model space. We will use a simple model with two excited states, with Woods-Saxon form factors on the diagonal, and Gaussian off the diagonal.

\begin{equation}
 H_{int} = 
 \begin{bmatrix}
   V(r) & \beta g(r) & \beta g(r) \\
  \beta g(r) & V(r) & \gamma g(r) \\
  \beta g(r) & \gamma g(r) & V(r) \\
 \end{bmatrix},
\end{equation}

where 

\begin{equation}
V(r) = \frac{V+iW}{1 + \exp{ \left( \frac{r-R}{a} \right) }},
\end{equation}

and 

\begin{equation}
g(r) = \exp{ \left( -\frac{r^2}{\alpha^2} \right) }.
\end{equation}

In this toy model, the real part of the Woods-Saxon in the $n$th diagonal represents the interaction of the projectile and target in the $n$th excited state, with $n=0$ being the ground state (and, therefore, the elastic channel). The imaginary component represents the transition from the $n$th channel to other excited states not explicitly included in the model space ($n$ > 2). The off-diagonal terms represent coupling between channels, with $\beta$ controlling the strength of coupling of the ground state to each excited state, and $\gamma$ between excited states.

The Schroedinger-like equation governing scattering in channel $n$ will look like [1, eq. (75)]:


\begin{equation}
\left( - \frac{\hbar^2}{2\mu}\nabla^2_{\vec r} + V(\vec r) - E_n \right) \psi_n(\vec r) = - \sum_{m \neq n} g_{mn}(\vec r) \psi_m(\vec r),
\end{equation}

where $E_n$ is the asymptotic kinetic energy in channel $n$ (total energy minus the energy of internal state $n$ of the system), and


\begin{equation}
g_{mn}(r) = \begin{cases} 
    \gamma g(r) & m,n = 0 \\
    \beta g(r) & \text{else}
\end{cases}
\end{equation}

In the case that $\gamma = 0$, this is equivalent to the distorted-wave Born approximation for the excited channels. If we turn on coupling between states with $\gamma \neq 0$, then this is a full coupled-channels calculation.


If the potentials are central and do not couple the particles spins, the projectile-target relative orbital angular momentum, $l$, is conserved and each wave-function can be expanded in partial waves as in ref. [1, eq. (36)]. It is then sufficient to solve independently the problem for each fixed value of $l$, reducing eq.(4) to


\begin{equation}
\left( - \frac{\hbar^2}{2\mu}\frac{d^2}{dr^2} + \frac{\hbar^2}{2\mu} \frac{\ell(\ell+1)}{r^2} + V(r) - E_n \right) \psi_{n,l}(r) = - \sum_{m \neq n} g_{mn}(r) \psi_{m,l}(r),
\end{equation}


[1]: Moro A. M., Models for nuclear reactions with weakly bound systems, https://arxiv.org/abs/1807.04349.

In the following, we will make the change of variables $s = k_n  r$, with the wavenumber in a channel $k_n$ in fm $^{-1}$ defined by $E_n \equiv \hbar^2 c^2 k_n^2 / (2 \mu)$, resulting in the following set of coupled equations:

\begin{equation}
 \left( -\frac{d^2}{ds^2} +  \frac{\ell(\ell+1)}{s^2} +  \frac{V(s)}{E_n}   - 1 \right) \psi_{n,l}(s) + \sum_{m \neq n}  \frac{g_{mn}(s)}{E_n} \psi_{m,l}(s) = 0,
\end{equation}

where $\mu$ is given in units of MeV / c $^2$.

It should be noted that our model describes scattering of a particle on a spinless target, with spinless excited states. If we set our ground or excited states to non-zero angular momenta, then partial waves ( $l$ states) would be coupled to eachother within and across energy levels, significantly increases the complexity of the system. We begin with such vastly simplified toy model to explore the construction of the reduced basis without confounding technical details.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.sparse import diags

In [29]:
# Constants
hbarc = 197 # MeV • fm
mass = 939 # MeV / c^2

# Potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R = 4  # Woods-Saxon potential radius
a = 0.5  # Woods-Saxon potential diffuseness

E_inc = 13.4   # bombarding energy in COM frame
l = 0      # Angular momentum quantum number

# strength of transition potentials MeV
beta = 10
gamma = 5

# range of transition potential
alpha = 5 # fm

# energy difference from groud state in each channel MeV
energy_levels = [0, 2, 4]

params_diag = (V0, W0, R, a)
params_transition = (alpha, beta, gamma)


def k_channel(En : float, E_inc : float):
    delta = En - E_inc
    return np.sqrt( 2 * mass * delta) / hbarc

In [50]:
def diag_potential(r : float, k : float, params: tuple):
    """
    Woods-Saxon with change of variable s = k * r
    """
    V, W, R, a = params
    E = k*k/mass
    s = r * k
    S = R * k
    d = a * k
    return -(V0 + W) / (1 + np.exp((s - S) / d)) / E
    
def transition_potential(r : float, k : float, params : tuple):
    """
    Gaussian with change of variable s = k * r
    """
    alpha, _ , _ = params
    E = k*k/mass
    d = alpha * k
    s = r * k
    return np.exp( -s**2 / d**2) / E

def full_diag_term(r : float, k : float,  l : int, params_diag : tuple):
    s = k * r
    return diag_potential(r, k, params_diag) + l * (l + 1) / s**2 - 1.0


def schrodinger_eqn(r, y, E_inc : float, l : int, energy_levels : list, params_diag : tuple, params_transition : tuple):
    """
    callback for scipy.integrate.solve_ivp. Takes in domain (r) and state (y)
    and returns dy/dr according to the RHS of the Schroedinger eqn.
    State (y) is the wavefunctions in each channel and their first derivatives, e.g.:
    [psi_0, psi_1, psi_2, psi_0', psi_1', psi_2']. 
    """
    alpha, beta, gamma = params_transition
    V, W, R, a = params_diag
    k = [k_channel(En, E_inc) for En in energy_levels ]
    
    psi = y[0:3]
    psi_prime = y[3:3*2]
    
    psi_double_prime = [
        # psi_0''
        full_diag_term(r, k[0], l, params_diag) * psi[0] 
      + beta * transition_potential(r, k[1] , params_transition) * psi[1] 
      + beta * transition_potential(r, k[2] , params_transition) * psi[2],
      # psi_1 ''
        beta * transition_potential(r, k[0] , params_transition) * psi[0] 
      + full_diag_term(r, k[1], l, params_diag) * psi[1] 
      + beta * transition_potential(r, k[2] , params_transition) * psi[2],
      # psi_2''
        beta * transition_potential(r, k[0] , params_transition) * psi[0] 
      + beta * transition_potential(r, k[1] , params_transition) * psi[1]
      + full_diag_term(r, k[2], l, params_diag) * psi[2] 
     ]

    return [ psi_prime[0] , psi_prime[1], psi_prime[2], psi_double_prime[0], psi_double_prime[1], psi_double_prime[2]] 

In [51]:
def solve_coupled_channels(E_inc : float, l : int, energy_levels : list,  params_diag : tuple, params_transition : tuple):
    alpha = params_transition
    V, W, R, a = params_diag

    # Integration parameters
    r_start = 0.05  # Initial position
    r_end=70  # Final position

    corr_factor=10

    # Initial conditions
    psi0 = corr_factor*r_start**(l+1) + 0j # Initial wavefunction value
    psi_prime0 = corr_factor*(l+1)*r_start**(l) +0j # Initial derivative value

    sol = solve_ivp(
        lambda r, y: schrodinger_eqn(r, y, E_inc, l, energy_levels, params_diag, params_transition), 
        (r_start, r_end), 
        [psi0, psi0, psi0, psi_prime0, psi_prime0, psi_prime0], 
        dense_output=True
    )

    return sol.sol[0:3]



In [52]:
# Integration parameters
r_start = 0.05  # Initial position
r_end=70  # Final position

corr_factor=10

# Initial conditions
psi0 = corr_factor*r_start**(l+1) + 0j # Initial wavefunction value
psi_prime0 = corr_factor*(l+1)*r_start**(l) +0j # Initial derivative value

# Solve the Schrödinger equation
sol = solve_coupled_channels(E_inc, l, energy_levels, params_diag, params_transition)

# Calculate the wavefunction
r_values = np.linspace(r_start, r_end, 1000)

# plot wavefunction in each channel
fig = plt.figure(dpi=200)
plt.plot(r_values, np.real(psi_values))
plt.plot(r_values, np.imag(psi_values))
plt.plot(r_values, potvals,'r')
plt.xlabel('r')
plt.ylabel('Psi(r)')
plt.show()

/tmp/ipykernel_1806/3656653670.py:30: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt( 2 * mass * delta) / hbarc
/home/codespace/.local/lib/python3.10/site-packages/scipy/integrate/_ivp/rk.py:109: RuntimeWarning: invalid value encountered in divide
  return norm(self._estimate_error(K, h) / scale)
